<a href="https://colab.research.google.com/github/KangHyeonGu1/-/blob/main/wellb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
data2=pd.read_csv('Wellbeing_and_lifestyle_data_Kaggle.csv')
data=pd.DataFrame({'supporting':data2['SUPPORTING_OTHERS'],'donation':data2['DONATION'],'personal_awards':data2['PERSONAL_AWARDS'],
                   'wlbs':data2['WORK_LIFE_BALANCE_SCORE']})
data.head


<bound method NDFrame.head of        supporting  donation  personal_awards   wlbs
0               0         0                4  609.5
1               8         2                3  655.6
2               4         2                4  631.6
3              10         5                5  622.7
4              10         4                8  663.9
...           ...       ...              ...    ...
15967           0         4                5  644.5
15968           7         3               10  714.9
15969          10         5               10  716.6
15970           2         2                1  682.0
15971          10         1                1  651.4

[15972 rows x 4 columns]>

In [7]:
# 독립변수 supporting 삼각퍼지화
su=data['supporting']
x1= su-0.5
x2= su+0.5
xdf=pd.DataFrame({'a':x1,'b':su,'c':x2})
xa=xdf[xdf['a']< 0]
xdf.loc[xa.index,'a']=0
x=xdf.to_numpy()

In [8]:
# 첫번째 매개변수 donation 삼각퍼지화
do=data['donation']
ma= do-0.25
mb= do+0.25
mdf=pd.DataFrame({'a':ma,'b':do,'c':mb})
ma=mdf[mdf['a']< 0]
mdf.loc[ma.index,'a']=0
m1=mdf.to_numpy()


In [9]:
# 두번째 매개변수 personal_awards 삼각퍼지화
pa=data['personal_awards']
ma2= pa-0.5
mb2= pa+0.5
m2df=pd.DataFrame({'a':ma2,'b':pa,'c':mb2})
ma2=m2df[m2df['a']< 0]
m2df.loc[ma2.index,'a']=0
m2=m2df.to_numpy()

In [10]:
# 종속변수 일과 삶의 균형점수 퍼지화 못한다고 생각. 
wl=data['wlbs']/10

ydf=pd.DataFrame({'a':wl,'b':wl,'c':wl})
y=ydf.to_numpy()


In [15]:
def dc(a,b): # 삼각퍼지 행렬곱 함수
  c=a*b
  sum=c.sum()
  return sum
xtx=np.zeros(shape=(2,2))
ones=np.ones(shape=(15972,3))
# 회귀계수 추정을 위한 삼각퍼지 x대치행렬과 x행렬 행렬곱 
xtx[0,0]=dc(ones,ones)
xtx[0,1]=dc(ones,x)
xtx[1,0] =dc(x,ones) 
xtx[1,1] =dc(x,x)
# m1= b0 + b1x 첫번째 매개변수 회귀계수 추정
xtm=np.zeros(shape=(2,1))
xtm[0,0]=dc(ones,m1)
xtm[1,0]=dc(x,m1)
lxtx=np.linalg.inv(xtx)
b1=lxtx@xtm
dc(ones,x)
b1
# m2 = b0 +b1x 두번째 매개변수 회귀계수 추정
xtm2=np.zeros(shape=(2,1))
xtm2[0,0]=dc(ones,m2)
xtm2[1,0]=dc(x,m2)
lxtx=np.linalg.inv(xtx)
b2=lxtx@xtm2
print(b1,b2)
# 추정결과 교수님 논문에 나온 퍼지 매개분석 결과와 마찬가지로 원래 추정치보다 양으로 더 높게 나옴

[[1.4547009 ]
 [0.22646111]] [[3.87769265]
 [0.32707942]]


In [13]:
xtx2=np.zeros(shape=(4,4))
xlist=[ones,x,m1,m2]
for i in range(0,4):   # 회귀계수 추정을 위한 삼각퍼지 x대치행렬과 x행렬 행렬곱 for문
  for j in range(0,4):
    xtx2[i,j]= dc(xlist[i],xlist[j])
#
xty=np.zeros(shape=(4,1))
for i in range(0,4):   # 회귀계수 추정을 위한 삼각퍼지 x대치행렬과 y행렬 행렬곱 for문
  for j in range(0,1):
    xty[i,j]= dc(xlist[i],y)
lxtx2=np.linalg.inv(xtx2)
bb=lxtx2@xty
bb
# 추정결과 일반 추정보다 조금 낮게 나옴. spread보완할 필요가 있음. spread 수정 생각하기

array([[59.85411709],
       [ 0.47709063],
       [ 0.55523871],
       [ 0.45877023]])